In [0]:
#cpu-used for general purpose task
#gpu-much more desirable for compute intensive task,has more cores and does more FLOPS than cpu.Also does parallel computation

In [5]:
import keras

Using TensorFlow backend.


# Data Preprocessing

Importing the libraries

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Importing the dataset

In [0]:
dataset=pd.read_csv("Churn_Modelling.csv")

In [8]:
dataset

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


Making the matrix of features , and seperating the dependent variable.

In [0]:
'''
for X we are using only two factors , 
age and salary to apply logistic regression and find out who purchases the product and who does not 
'''
X=dataset.iloc[:,3:13].values
y=dataset.iloc[:,-1].values

Encoding the categorical variables .

In [0]:

from sklearn.preprocessing import LabelEncoder,OneHotEncoder
labelencoder_X_1=LabelEncoder()
X[:,1]=labelencoder_X_1.fit_transform(X[:,1])
labelencoder_X_2=LabelEncoder()
X[:,2]=labelencoder_X_2.fit_transform(X[:,2])
# so we get the encoded values but there might be problems, that is why we do dummy coding

In [20]:
X


array([[619, 0, 0, ..., 1, 1, 101348.88],
       [608, 2, 0, ..., 0, 1, 112542.58],
       [502, 0, 0, ..., 1, 0, 113931.57],
       ...,
       [709, 0, 0, ..., 0, 1, 42085.58],
       [772, 1, 1, ..., 1, 0, 92888.52],
       [792, 0, 0, ..., 1, 0, 38190.78]], dtype=object)

Dummy coding and removing one dummy coded column to avoid dummy variable trap

In [0]:
#Dummy coding using OneHotEncoder
#X
from sklearn.compose import ColumnTransformer

transformer = ColumnTransformer(
    transformers=[
        ("OneHot",        # Just a name
         OneHotEncoder(), # The transformer class
         [1]              # The column(s) to be applied on.
         )
    ],
    remainder='passthrough' # donot apply anything to the remaining columns
)
X = transformer.fit_transform(X.tolist())
X = X.astype('float64')

# To avoid the dummy variable trap , we remove one  of the dummy variables 
X=X[:,1:]

In [26]:
X

array([[0.0000000e+00, 0.0000000e+00, 6.1900000e+02, ..., 1.0000000e+00,
        1.0000000e+00, 1.0134888e+05],
       [0.0000000e+00, 1.0000000e+00, 6.0800000e+02, ..., 0.0000000e+00,
        1.0000000e+00, 1.1254258e+05],
       [0.0000000e+00, 0.0000000e+00, 5.0200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 1.1393157e+05],
       ...,
       [0.0000000e+00, 0.0000000e+00, 7.0900000e+02, ..., 0.0000000e+00,
        1.0000000e+00, 4.2085580e+04],
       [1.0000000e+00, 0.0000000e+00, 7.7200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 9.2888520e+04],
       [0.0000000e+00, 0.0000000e+00, 7.9200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 3.8190780e+04]])

Splitting dataset into training set and test set

In [0]:
#splitting the dataset into a training set and a test set
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=0)

Applying feature scaling  to ease computation

In [0]:
#feature scaling(Compulsary for neural networks )
from sklearn.preprocessing import StandardScaler
sc_X=StandardScaler()
X_train=sc_X.fit_transform(X_train)
X_test=sc_X.transform(X_test)
#y consists of categorical values so we do not need to scale it

In [0]:
import keras

In [32]:
X_train

array([[-0.5698444 ,  1.74309049,  0.16958176, ...,  0.64259497,
        -1.03227043,  1.10643166],
       [ 1.75486502, -0.57369368, -2.30455945, ...,  0.64259497,
         0.9687384 , -0.74866447],
       [-0.5698444 , -0.57369368, -1.19119591, ...,  0.64259497,
        -1.03227043,  1.48533467],
       ...,
       [-0.5698444 , -0.57369368,  0.9015152 , ...,  0.64259497,
        -1.03227043,  1.41231994],
       [-0.5698444 ,  1.74309049, -0.62420521, ...,  0.64259497,
         0.9687384 ,  0.84432121],
       [ 1.75486502, -0.57369368, -0.28401079, ...,  0.64259497,
        -1.03227043,  0.32472465]])

# Building the neural network

In [0]:
from keras.models import Sequential
from keras.layers import Dense

Initialising the ANN

In [0]:
#initialising the ANN
#this neural network is a classifier 
classifier=Sequential()


Designing the neural network

In [37]:
#Adding the input layer and hidden layer 
#parameter tuning (k-fold cross validation)
classifier.add(Dense(output_dim=6,init='uniform',activation='relu',input_dim=11))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=11, units=6, kernel_initializer="uniform")`
  """Entry point for launching an IPython kernel.


In [38]:
#Adding another hidden layer
classifier.add(Dense(output_dim=6,init='uniform',activation='relu'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=6, kernel_initializer="uniform")`
  """Entry point for launching an IPython kernel.


In [39]:
#adding output layer, using sigmoid or softmax as activation function
classifier.add(Dense(output_dim=1,init='uniform',activation='sigmoid'))
#if output categories are 3, then output_dim is 3 and activation function is softmax


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  """Entry point for launching an IPython kernel.


In [0]:
#Applying SGD on the ANN and compiling it 
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])#metrics are used to improve the model

Training the neural network

In [42]:
classifier.fit(X_train,y_train,batch_size=10,epochs= 100)


Epoch 1/100
8000/8000 [==============================] - 1s 159us/step - loss: 0.4881 - accuracy: 0.7958
Epoch 2/100
8000/8000 [==============================] - 1s 105us/step - loss: 0.4269 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 104us/step - loss: 0.4213 - accuracy: 0.8004
Epoch 4/100
8000/8000 [==============================] - 1s 105us/step - loss: 0.4176 - accuracy: 0.8234
Epoch 5/100
8000/8000 [==============================] - 1s 113us/step - loss: 0.4156 - accuracy: 0.8267
Epoch 6/100
8000/8000 [==============================] - 1s 111us/step - loss: 0.4138 - accuracy: 0.8310
Epoch 7/100
8000/8000 [==============================] - 1s 106us/step - loss: 0.4123 - accuracy: 0.8321
Epoch 8/100
8000/8000 [==============================] - 1s 106us/step - loss: 0.4115 - accuracy: 0.8324
Epoch 9/100
8000/8000 [==============================] - 1s 104us/step - loss: 0.4102 - accuracy: 0.8319
Epoch 10/100
8000/8000 [==============================]

# Making Predictions

In [44]:
# making the final predictions
y_pred=classifier.predict(X_test)
y_pred

array([[0.23821467],
       [0.37632686],
       [0.19036964],
       ...,
       [0.20995718],
       [0.18861622],
       [0.12805504]], dtype=float32)

### **Conclusion:**Each entry in the array y_pred represents the probability of a person leaving the bank, example, first person has 24% of leaving the bank.The higher the probability the higher are the chances that the person has left the bank . We can also calculate the accuracy of the model on the test set using confusion matrix.